# Correlation between functional selections on humanDAG1 and mastomysDAG1 cells

Compare functional scores measured on cells expressing humanDAG1 and mastomysDAG1

In [ ]:
# Imports
import pandas as pd
import seaborn as sns
import numpy as np
import scipy as sp
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import altair as alt


# Seaborn style settings
sns.set(rc={"figure.dpi" : 300})
sns.set_style("ticks")

# Allow more rows for Altair
_ = alt.data_transformers.disable_max_rows()

File paths for data:

In [ ]:
humanDAG1_data_path = "results/func_effects/averages/human_293T_entry_func_effects.csv"
mastomysDAG1_data_path = "results/func_effects/averages/mastomys_293T_entry_func_effects.csv"

In [ ]:
# Read data
human_df = pd.read_csv(humanDAG1_data_path)
mastomys_df = pd.read_csv(mastomysDAG1_data_path)

# Merge data on intersection of measured values
merged_df = (
    human_df.merge(
        mastomys_df,
        how="inner",
        on=["site", "wildtype", "mutant"],
        suffixes=["_human", "_mastomys"],
        validate="one_to_one",
    )
)

# Add average times seen column
merged_df["average_times_seen"] = merged_df[["times_seen_human", "times_seen_mastomys"]].mean(axis=1)

merged_df.head()

Plot correlation of scores with an interactive plot

In [ ]:
r, p = sp.stats.pearsonr(merged_df["effect_human"], merged_df["effect_mastomys"])
print(f"r correlation with no filters applied: {r:.2f}")
print(f"r^2 correlation with no filters applied: {r**2:.2f}")

slider = alt.binding_range(min=1, max=25, step=1, name="times_seen")
selector = alt.param(name="SelectorName", value=1, bind=slider)

alt.Chart(merged_df).mark_point(filled=True, color="black").encode(
    alt.X(
        "effect_human",
        axis=alt.Axis(
            title="functional score measured in humanDAG1 expressing cells", 
            titleFontSize=12,
            values=[-6, -4, -2, 0, 2],
            domainWidth=1,
            domainColor="black",
            tickColor="black",
        ),
        scale=alt.Scale(domain=[-6,2])
    ),
    alt.Y(
        "effect_mastomys",
        axis=alt.Axis(
            title="functional score measured in mastomysDAG1 expressing cells", 
            titleFontSize=12,
            values=[-6, -4, -2, 0, 2],
            domainWidth=1,
            domainColor="black",
            tickColor="black",
        ),
        scale=alt.Scale(domain=[-6,2])
    ),
    opacity=alt.condition(
        alt.datum.average_times_seen < selector,
        alt.value(0.1),
        alt.value(1)
    ),
    tooltip=[
        "site",
        "wildtype",
        "mutant",
        "effect_human",
        "times_seen_human",
        "n_selections_human",
        "effect_mastomys",
        "times_seen_mastomys",
        "n_selections_mastomys"
    ],
).add_params(
   selector
).configure_axis(
    grid=False
).properties(
    width=500,
    height=500
)